In [91]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import Inspector
from datetime import datetime as dt
import numpy as np
from config import userlogin

### Extract CSVs into DataFrame

In [5]:
csv_file = "Resources/full_data.csv"
full_data_df = pd.read_csv(csv_file)
full_data_df.head(None)

,date,location,new_cases,new_deaths,total_cases,total_deaths,weekly_cases,weekly_deaths,biweekly_cases,biweekly_deaths
0,2020-02-24,Afghanistan,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,2020-02-25,Afghanistan,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,2020-02-26,Afghanistan,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,2020-02-27,Afghanistan,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,2020-02-28,Afghanistan,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
89957,2021-05-28,Zimbabwe,64.0,0.0,38918.0,1592.0,254.0,6.0,383.0,10.0
89958,2021-05-29,Zimbabwe,15.0,2.0,38933.0,1594.0,254.0,8.0,379.0,12.0
89959,2021-05-30,Zimbabwe,11.0,0.0,38944.0,1594.0,262.0,8.0,384.0,12.0
89960,2021-05-31,Zimbabwe,17.0,0.0,38961.0,1594.0,265.0,8.0,389.0,12.0


In [6]:
world_2021_file = "Resources/world-happiness-report-2021.csv"
world_2021_df = pd.read_csv(world_2021_file)
world_2021_df.head(None)
#Need to add year as column with rows indicating 2021
#Filter by US only
#Columns will be: Healthy Life Expectency, Freedom, 

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.000,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.700,0.946,0.030,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.400,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.000,0.955,0.160,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.400,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Lesotho,Sub-Saharan Africa,3.512,0.120,3.748,3.276,7.926,0.787,48.700,0.715,-0.131,0.915,2.43,0.451,0.731,0.007,0.405,0.103,0.015,1.800
145,Botswana,Sub-Saharan Africa,3.467,0.074,3.611,3.322,9.782,0.784,59.269,0.824,-0.246,0.801,2.43,1.099,0.724,0.340,0.539,0.027,0.088,0.648
146,Rwanda,Sub-Saharan Africa,3.415,0.068,3.548,3.282,7.676,0.552,61.400,0.897,0.061,0.167,2.43,0.364,0.202,0.407,0.627,0.227,0.493,1.095
147,Zimbabwe,Sub-Saharan Africa,3.145,0.058,3.259,3.030,7.943,0.750,56.201,0.677,-0.047,0.821,2.43,0.457,0.649,0.243,0.359,0.157,0.075,1.205


In [7]:
world_report_file = "Resources/world-happiness-report.csv"
world_report_df = pd.read_csv(world_report_file)
world_report_df.head()
#Filter just 2020 and 2021


,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268


### Create new data with select columns

In [87]:

#Select columns, create copy
new_full_data_df = full_data_df[['location','total_cases','total_deaths']].copy()
#Select country? or just match all data to dataset with least amount of countries

#new_full_data_df.loc[(new_full_data_df.location == 'United States')]
#Set NaN for total_deaths as 0
new_full_data_df = new_full_data_df.fillna(0)
#Change exact date to year
new_full_data_df['year'] = pd.DatetimeIndex(full_data_df['date'])
#Set index to year

grouped_full_data_df = new_full_data_df.groupby('location')
grouped_full_data_df2 = grouped_full_data_df.agg(max_date=('year', np.max), min_date=('year', np.min))



#Select only most recent year for 2021
new_full_data_df21 = new_full_data_df.loc[new_full_data_df['year'] == '2021-06-01'].copy()
new_full_data_df21['year'] = pd.DatetimeIndex(new_full_data_df21['year']).year
new_full_data_df21.set_index('year', inplace=True)
new_full_data_df21 = new_full_data_df21[new_full_data_df21.location != 'World']
new_full_data_df21.head(None)
#Select equivalent date for 2020
new_full_data_df20 = new_full_data_df.loc[new_full_data_df['year'] == '2020-06-01'].copy()
new_full_data_df20['year'] = pd.DatetimeIndex(new_full_data_df20['year']).year
new_full_data_df20.set_index('year', inplace=True)
new_full_data_df20 = new_full_data_df20[new_full_data_df20.location != 'World']
new_full_data_df20.head(None)



,location,total_cases,total_deaths
year,,,
2020,Afghanistan,15753.0,266.0
2020,Africa,152344.0,4343.0
2020,Albania,1143.0,33.0
2020,Algeria,9513.0,661.0
2020,Andorra,765.0,51.0
...,...,...,...
2020,Venezuela,1662.0,17.0
2020,Vietnam,328.0,0.0
2020,Yemen,354.0,84.0


In [69]:
#World Happiness for 2021
#Select columns
world_2021_cols = ['Country name', 'Healthy life expectancy', 'Freedom to make life choices', 'Perceptions of corruption']
transformed_2021 = world_2021_df[world_2021_cols].copy()
#Rename columns for sql
transformed_2021 = transformed_2021.rename(columns={'Country name': 'country','Healthy life expectancy': 'life_expectency',
    'Freedom to make life choices': 'life_freedom',
    'Perceptions of corruption': 'corruption_perception'})
#Fill missing year index/column set to 2021
transformed_2021['year'] = 2021
transformed_2021.set_index('year', inplace=True)
transformed_2021 = transformed_2021.sort_values(by='country')
transformed_2021.head(None)

,country,life_expectency,life_freedom,corruption_perception
year,,,,
2021,Afghanistan,52.493,0.382,0.924
2021,Albania,68.999,0.785,0.901
2021,Algeria,66.005,0.480,0.752
2021,Argentina,69.000,0.828,0.834
2021,Armenia,67.055,0.825,0.629
...,...,...,...,...
2021,Venezuela,66.700,0.615,0.827
2021,Vietnam,68.034,0.940,0.796
2021,Yemen,57.122,0.602,0.800


In [81]:
#World Happiness Report
#Select columns
world_report_cols = ['year', 'Country name', 'Healthy life expectancy at birth', 'Freedom to make life choices', 'Perceptions of corruption']
#Create copy
transformed_world_report = world_report_df[world_report_cols].copy()
#Rename Columns
transformed_world_report = transformed_world_report.rename(columns={'year': 'year','Country name': 'country','Healthy life expectancy at birth': 'life_expectency',
    'Freedom to make life choices': 'life_freedom',
    'Perceptions of corruption': 'corruption_perception'})
#Set index to year
transformed_world_report.set_index('year', inplace=True)
#Select only 2020 year
transformed_world_report = transformed_world_report.loc[[2020]]
transformed_world_report.head(None)

,country,life_expectency,life_freedom,corruption_perception
year,,,,
2020,Albania,69.3,0.754,0.891
2020,Argentina,69.2,0.823,0.816
2020,Australia,74.2,0.905,0.491
2020,Austria,73.6,0.912,0.464
2020,Bahrain,69.7,0.945,NaN
...,...,...,...,...
2020,United States,68.1,0.850,0.678
2020,Uruguay,69.2,0.908,0.491
2020,Venezuela,66.9,0.612,0.811


### Create database connection

In [92]:
rds_connection_string = (f"{userlogin}@localhost:5432/customer_db")
engine = create_engine(f'postgresql://{rds_connection_string}')

In [94]:
#Confirm tables
Inspector.get_table_names()

NameError: name 'Inspector' is not defined